# Rf 4 0 1_Importttreethx
Data and categories: advanced options for importing data from ROOT TTree and THx histograms

Basic import options are demonstrated in rf102_dataimport.C



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:51 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooDataHist.h"
#include "RooCategory.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"
#include "TH1.h"
#include "TTree.h"
#include "TRandom.h"
#include <map>

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

In [3]:
TH1 *makeTH1(const char *name, Double_t mean, Double_t sigma);
TTree *makeTTree();

input_line_28:3:17: warning: empty parentheses interpreted as a function declaration [-Wvexing-parse]
TTree *makeTTree();
                ^~
input_line_28:3:17: note: replace parentheses with an initializer to declare a variable
TTree *makeTTree();
                ^~
                 = nullptr


 A helper function is created: 

In [4]:
%%cpp -d
TH1 *makeTH1(const char *name, Double_t mean, Double_t sigma)
{
   // Create ROOT TH1 filled with a Gaussian distribution

   TH1D *hh = new TH1D(name, name, 100, -10, 10);
   for (int i = 0; i < 1000; i++) {
      hh->Fill(gRandom->Gaus(mean, sigma));
   }
   return hh;
}

 A helper function is created: 

In [5]:
%%cpp -d
TTree *makeTTree()
{
   // Create ROOT TTree filled with a Gaussian distribution in x and a uniform distribution in y

   TTree *tree = new TTree("tree", "tree");
   Double_t *px = new Double_t;
   Double_t *py = new Double_t;
   Double_t *pz = new Double_t;
   Int_t *pi = new Int_t;
   tree->Branch("x", px, "x/D");
   tree->Branch("y", py, "y/D");
   tree->Branch("z", pz, "z/D");
   tree->Branch("i", pi, "i/I");
   for (int i = 0; i < 100; i++) {
      *px = gRandom->Gaus(0, 3);
      *py = gRandom->Uniform() * 30 - 15;
      *pz = gRandom->Gaus(0, 5);
      *pi = i % 3;
      tree->Fill();
   }
   return tree;
}

Import multiple th1 into a roodatahist
 --------------------------------------------------------------------------

Create thee root th1 histograms

In [6]:
TH1 *hh_1 = makeTH1("hh1", 0, 3);
TH1 *hh_2 = makeTH1("hh2", -3, 1);
TH1 *hh_3 = makeTH1("hh3", +3, 4);

Declare observable x

In [7]:
RooRealVar x("x", "x", -10, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create category observable c that serves as index for the root histograms

In [8]:
RooCategory c("c", "c");
c.defineType("SampleA");
c.defineType("SampleB");
c.defineType("SampleC");

Create a binned dataset that imports contents of all th1 mapped by index category c

In [9]:
RooDataHist *dh = new RooDataHist("dh", "dh", x, Index(c), Import("SampleA", *hh_1), Import("SampleB", *hh_2),
                                  Import("SampleC", *hh_3));
dh->Print();

RooDataHist::dh[c,x] = 300 bins (2964 weights)


Alternative constructor form for importing multiple histograms

In [10]:
map<string, TH1 *> hmap;
hmap["SampleA"] = hh_1;
hmap["SampleB"] = hh_2;
hmap["SampleC"] = hh_3;
RooDataHist *dh2 = new RooDataHist("dh", "dh", x, c, hmap);
dh2->Print();

RooDataHist::dh[c,x] = 300 bins (2964 weights)


Importing a ttree into a roodataset with cuts
 -----------------------------------------------------------------------------------------

In [11]:
TTree *tree = makeTTree();

Define observables y,z

In [12]:
RooRealVar y("y", "y", -10, 10);
RooRealVar z("z", "z", -10, 10);

Import only observables (y,z)

In [13]:
RooDataSet ds("ds", "ds", RooArgSet(x, y), Import(*tree));
ds.Print();

[#1] INFO:Eval -- RooTreeDataStore::loadValues(ds) Ignored 35 out of range events
RooDataSet::ds[x,y] = 65 entries


Import observables (x,y,z) but only event for which (y+z<0) is true

In [14]:
RooDataSet ds2("ds2", "ds2", RooArgSet(x, y, z), Import(*tree), Cut("y+z<0"));
ds2.Print();

[#1] INFO:Eval -- RooTreeDataStore::loadValues(ds2) Ignored 36 out of range events
RooDataSet::ds2[x,y,z] = 26 entries


Importing integer ttree branches
 ---------------------------------------------------------------

Import integer tree branch as roorealvar

In [15]:
RooRealVar i("i", "i", 0, 5);
RooDataSet ds3("ds3", "ds3", RooArgSet(i, x), Import(*tree));
ds3.Print();

[#1] INFO:Eval -- RooAbsReal::attachToTree(i) TTree Int_t branch i will be converted to double precision
RooDataSet::ds3[i,x] = 100 entries


Define category i

In [16]:
RooCategory icat("i", "i");
icat.defineType("State0", 0);
icat.defineType("State1", 1);

Import integer tree branch as roocategory (only events with i==0 and i==1
 will be imported as those are the only defined states)

In [17]:
RooDataSet ds4("ds4", "ds4", RooArgSet(icat, x), Import(*tree));
ds4.Print();

[#1] INFO:DataHandling -- RooAbsCategory::attachToTree(i) TTree branch i will be interpreted as category index
[#1] INFO:Eval -- RooTreeDataStore::loadValues(ds4) Ignored 33 out of range events
RooDataSet::ds4[i,x] = 67 entries


Import multiple roodatasets into a roodataset
 ----------------------------------------------------------------------------------------

Create three roodatasets in (y,z)

In [18]:
RooDataSet *dsA = (RooDataSet *)ds2.reduce(RooArgSet(x, y), "z<-5");
RooDataSet *dsB = (RooDataSet *)ds2.reduce(RooArgSet(x, y), "abs(z)<5");
RooDataSet *dsC = (RooDataSet *)ds2.reduce(RooArgSet(x, y), "z>5");

Create a dataset that imports contents of all the above datasets mapped by index category c

In [19]:
RooDataSet *dsABC = new RooDataSet("dsABC", "dsABC", RooArgSet(x, y), Index(c), Import("SampleA", *dsA),
                                   Import("SampleB", *dsB), Import("SampleC", *dsC));

dsABC->Print();

RooDataSet::dsABC[c,x,y] = 26 entries
